In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

years = np.array([1979, 1982, 1985, 1988, 1991, 1994, 1997, 2000, 2003, 2006, 2009, 2012, 2015, 2016], dtype=np.float64)
ice_areas = np.array([7.00, 7.34, 6.79, 7.47, 6.46, 7.15, 6.85, 6.37, 6.21, 5.97, 5.31, 3.40, 4.59, 4.30], dtype=np.float64)

# these are the points we're asked to predict at some point in the exercises
years_to_predict = np.array([1984, 1999, 2017, 2018, 2019], dtype=np.float64)
ice_areas_actual = np.array([6.8, 6.12, 4.6, 4.8, 4.32], dtype=np.float64)

In [ ]:
slope, intercept, r_Value, p_value, std_err = stats.linregress(years, ice_areas)

In [ ]:
# use same time range everywhere
time_steps = np.linspace(1978, 2020, 100000)

plt.figure(figsize=(7,7))
plt.title("Linear regression")
plt.ylabel("Ice surface (million square km)")
plt.xlabel("Year")

plt.scatter(years, ice_areas, label='Data points')
plt.scatter(years_to_predict, ice_areas_actual, label="Actual data points (we need to predict)")

plt.plot(time_steps, [slope * timestep + intercept for timestep in time_steps], label="Linear regression")

plt.ylim(0, 10)
plt.legend()

print("Mean absolute error: ", mean_absolute_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))
print("Mean squared error: ", mean_squared_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))

## Exercise 2
For constructing a polynomial interpolating function through the data, consider the following possible sets of basis functions: (a) the set $\phi_j(t)=t^{j-1}$ (these are the regular monomials), (b) the set $\phi_j(t)=(t-1978)^{j-1}$, (c) the set $\phi_j(t)=((t-1998)/18)^{j-1}$. The index $j$ runs from 1 to $J$.

What should be the value of $J$? For each of the sets of basis functions (a), (b), (c), compute the corresponding Vandermonde matrix. How do their condition numbers compare?

In [ ]:
# fill with ones (because the first column is always ones)
phi1 = np.ones((len(years),len(years)))
phi2 = np.ones((len(years),len(years)))
phi3 = np.ones((len(years),len(years)))

# set up Vandermonde matrix for each basis function
for i in range(0, len(years)):
    for j in range(0, len(years)):
        phi1[i][j] = years[i]**j
        phi2[i][j] = (years[i] - 1978)**j 
        phi3[i][j] = ((years[i] - 1998)/18)**j 
        
phi1pd = pd.DataFrame(phi1)
        
print("Condition number of basis (a) is: ", np.linalg.cond(phi1))
print("Condition number of basis (b) is: ", np.linalg.cond(phi2))
print("Condition number of basis (c) is: ", np.linalg.cond(phi3))

#### Discussion of exercise 2
The values of J should run from 1 to length(input) where the input is the dates data.

The condition number of basis (c) is much better conditioned than that of condition (a) or (b).  Though (c) is still not very well condtioned, meaning that it will be sensitive towards small changes in the input.

## Exercise 3
Use the set of basis functions with the best conditioning and compute the interpolating function through the data in the table above. Show the coefficients of the interpolating function in a table. Plot the data points and the interpolating function in a single figure. Discuss the results.

What values does this interpolating function predict for $t=1984$? What values for $t=2017$ and $t=2018$? Check the NASA website mentioned in the introduction to see how good these predictions are.

In [ ]:
# calculate coefficients for the function
coefficients = np.linalg.solve(phi3, ice_areas)

In [ ]:
# set up interpolation function
def get_function_value(time_step, c):
    x = ((time_step-1998)/18)
    
    y = c[13] * x**13 + c[12] * x**12 + c[11] * x**11 + c[10] * x**10 + c[9] * x**9 + c[8] * x**8 +\
        c[7] * x**7 + c[6] * x**6 + c[5] * x**5 + c[4] * x**4 + c[3] * x**3 + c[2] * x**2 + c[1] * x + c[0]
    
    return y

In [ ]:
interpolation_ice_area = [get_function_value(time_step, coefficients) for time_step in time_steps]

# Plot it!
plt.figure(figsize=(7,7))
plt.title("Interpolation with basis set (c)")
plt.ylabel("Ice surface (million square km)")
plt.xlabel("Year")

plt.scatter(years, ice_areas, label='Data points')
plt.plot(time_steps, interpolation_ice_area, label='Interpolated function')
plt.plot(time_steps, [slope * timestep + intercept for timestep in time_steps], label="Linear regression")
plt.scatter(years_to_predict, ice_areas_actual, label="Actual data points (we need to predict)", color="red")

plt.ylim(0, 25)
plt.legend()

print("Mean absolute error linear: ", mean_absolute_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))
print("Mean squared error linear: ", mean_squared_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))
print("Mean absolute error interpolation: ", mean_absolute_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))
print("Mean squared error interpolation: ", mean_squared_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))

## Exercise 5
As an alternative to interpolation, one can use minimization (e.g. linear least squares) to fit a function to the data. Use linear least squares to fit a low-degree (e.g., quadratic or cubic) polynomial to the data in the table. Plot the resulting function together with the data. Make predictions for values at $t=1984$, $t=2017$ and $t=2018$. Compare the results to the results obtained before with interpolation.

Explore the sensitivity of the least squares fit to changes in the data, in the same way as before.

In [ ]:
def quadratic_polynomial(x, coefficients):
    return coefficients[0]*x**2 + coefficients[1]*x + coefficients[2]

# Design the matrices
Qm = np.array([[T**2, T, 1] for T in years])

In [ ]:
# Quadratic function fitting

# Find the parameters (grab zero'th element because these are the coefficients)
qp = np.linalg.lstsq(Qm, ice_areas, rcond=None)[0]

# Calculate the line with the parameters 
yQ = quadratic_polynomial(time_steps, qp)

In [ ]:
# Plot both fittings 
plt.figure(figsize=(7,7))
plt.title("Different regression methods")
plt.ylabel("Ice surface (million square km)")
plt.xlabel("Year")

# datapoints
plt.scatter(years, ice_areas, label='Data points')
plt.scatter(years_to_predict, ice_areas_actual, label="Actual data points (we need to predict)", color="red")

# linear
plt.plot(time_steps, [slope * timestep + intercept for timestep in time_steps], label="Linear regression")

# interpolation
plt.plot(time_steps, interpolation_ice_area, label='Interpolated function')

# least squares quadratic
plt.plot(time_steps, yQ, label='Least squares quadratic polynomial')

plt.ylim(0 , 10)
plt.legend()
plt.show()

print("Mean absolute error linear: ", mean_absolute_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))
print("Mean squared error linear: ", mean_squared_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))

print("Mean absolute error interpolation: ", mean_absolute_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))
print("Mean squared error interpolation: ", mean_squared_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))

print("Mean absolute ls quadratic: ", mean_absolute_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))
print("Mean squared ls quadratic: ", mean_squared_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))

## Exercise 7
There may be more suitable functions to fit to these data than a polynomial. For example, predictions using a polynomial may give negative values, which would be unphysical. Can you think of a suitable function, and fit it to the data? What predictions does it give? If the function is nonlinear in its parameters, fitting it by least squares will give a _nonlinear least squares_ problem (see section 6.6 in the book by Heath). 

Three functions that may be more suitable functions to fit to these data are:
the exponential function, the sigmoid function, and the cosine function. 
Regarding the plot of the datapoints, intuitively the general trend seems to have the form of the 
exponential function at first glance. The sigmoid function seems applicable too. 
The cosine function makes sense to experiment with as well, as the topic is related to the global 
temperatures, and climates can change back and forth, following a general trend like that of 
the cosine function. 
The Levenberg-Marquardt algorithm was used to fit these three functions. 
For all functions, experiments were run with different initial estimates for the parameters.

In [ ]:
from scipy import interpolate
from scipy import optimize
import scipy as sp
from scipy.optimize import leastsq 
from scipy.optimize import curve_fit
from math import *


In [ ]:
tck = sp.interpolate.splrep(years, ice_areas, s=0)
ynew = interpolate.splev(time_steps, tck, der=0)


plt.figure(figsize=(7,7))
plt.title("Least squares polynomials")
plt.ylabel("Ice surface (million square km)")
plt.xlabel("Year")

# datapoints
plt.scatter(years, ice_areas, label='Data points')
plt.scatter(years_to_predict, ice_areas_actual, label="Actual data points (we need to predict)", color="red")

# linear
plt.plot(time_steps, [slope * timestep + intercept for timestep in time_steps], label="Linear regression")

# interpolation
plt.plot(time_steps, interpolation_ice_area, label='Interpolated function')

# least squares quadratic
plt.plot(time_steps, yQ, label='Least squares quadratic polynomial')

# cubic spline
plt.plot(time_steps, ynew, label="Cubic splines")

plt.ylim(0 , 10)
plt.legend()
plt.show()


print("Mean absolute error linear: ", mean_absolute_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))
print("Mean squared error linear: ", mean_squared_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))

print("Mean absolute error interpolation: ", mean_absolute_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))
print("Mean squared error interpolation: ", mean_squared_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))

print("Mean absolute ls quadratic: ", mean_absolute_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))
print("Mean squared ls quadratic: ", mean_squared_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))

print("Mean absolute cubic splines: ", mean_absolute_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))
print("Mean squared cubic splines: ", mean_squared_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))

The cubic spline fit can be seen to fit the data points well, but the splines before and after the data points does not seem to follow the trend.  If compared to the known points for 1984 and 2016-2018, it can be seen that the spline matches the point for 1984 pretty good, but does not match the rest of the data.

The cosine function seems to fit well, with the fits only making sense for initial estimates of a being 0, 0.05 and 0.1. Higher than 0.1 results in an off fit, as can be seen in the example with a = 0.2.

In [ ]:
# fit a simple cosine function to the data
def cosine_function(x, a, b, c, d):
    return a * np.cos(b*(c + x)) + d

In [ ]:
ps = 0.05

import seaborn as sns
sns.set()

params = optimize.curve_fit(cosine_function, years, ice_areas, p0=[2, ps, 0, 5])[0]

print("Amplitude, period, horizontal move , equilibrium\n", params)


plt.figure(figsize=(7,7))
plt.title("Fits with different regression methods")
plt.ylabel("Ice surface (million square km)")
plt.xlabel("Year")

# datapoints
plt.scatter(years, ice_areas, label='Data points')
plt.scatter(years_to_predict, ice_areas_actual, label="Actual data points (we need to predict)", color="red")

# linear
plt.plot(time_steps, [slope * timestep + intercept for timestep in time_steps], label="Linear regression")

# interpolation
plt.plot(time_steps, interpolation_ice_area, label='Interpolated function')

# least squares quadratic
plt.plot(time_steps, yQ, label='Least squares quadratic polynomial')

# cosine fit
plt.plot(time_steps, cosine_function(time_steps, params[0], params[1], params[2], params[3]), label="Cosine curve fit")

plt.ylim(2, 10)
plt.legend()
plt.show()

plt.savefig("results/regression.png", dpi=1000)

print("Mean absolute error linear: ", mean_absolute_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))
print("Mean squared error linear: ", mean_squared_error(ice_areas_actual, [slope * year + intercept for year in years_to_predict]))

print("Mean absolute error interpolation: ", mean_absolute_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))
print("Mean squared error interpolation: ", mean_squared_error(ice_areas_actual, [get_function_value(year, coefficients) for year in years_to_predict]))

print("Mean absolute ls quadratic: ", mean_absolute_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))
print("Mean squared ls quadratic: ", mean_squared_error(ice_areas_actual, [quadratic_polynomial(year, qp) for year in years_to_predict]))

print("Mean absolute cosine: ", mean_absolute_error(ice_areas_actual, [cosine_function(year, params[0], params[1], params[2], params[3]) for year in years_to_predict]))
print("Mean squared cosine: ", mean_squared_error(ice_areas_actual, [cosine_function(year, params[0], params[1], params[2], params[3]) for year in years_to_predict]))

From the plots for the cosine function it can be seen that with the LM method the initial estimates can be very important. In practice, the estimates should be based on more than trial and error. This is not unique to the cosine function, the same behavior was observed with the other functions. We have simply chosen to keep the plot of the erroneous fitting of the cosine function as an example for this point.

Both of the cosine function can't get negative, but the first one (for a = 0.2) doesn't fit the data well. 

As in the sigmoid graph the initial values are once again an issue, since the start values are not close enough to the actual values, they do not converge and the function is poorly fit. This function cannot be negative either, but does not fit the data well.


The exponential function seems to fit the data well, as expected at first glance on the datapoints. A possible tweak to improve the fit would be to experiment with higher initial values for parameter d, closer to 3 to be precise, to try and create a sigmoid type function.


In [ ]:
# fitting function: p * -e^b*(x + c)^d, want multiple parameters to fit
def pval(x, p, b, c, d):
    return p*-np.exp(b*(x+c)**d)

# initial guess for the parameters: don't really matter
p_0 = (5, -0.0001, -1960, 2.5)

p0 = np.array([p_0])

# Curve fit calls leastsq under the hood and uses Levenberg-Marquardt
plsq = curve_fit(pval, years, ice_areas, p0=p_0, maxfev=10000000)#, method='lm')

# Create the actual line
vals = plsq[0][0]*-np.exp(plsq[0][1]*(years+plsq[0][2])**plsq[0][3])

plt.scatter(years, ice_areas, label='Original normalized data')
plt.plot(years, vals, label='Fitted data exponential function: p * -e^b*(x + c)^d')
plt.legend()
plt.title("Exponential Fit Function: p * -e^b*(x + c)^d")
plt.ylabel("Ice surface (million square km)")
plt.xlabel("Year")
plt.show()

#print(plsq)

The predictions obtained with the cosine and exponential functions show expected trends. The cosine function shows an increase of ice areas in 2018, whereas the exponential function expects further decrease. 

From our earlier findings on the effect of the initial estimates for parameters and the predictions (which support what we have referred to earlier as a general 'form of the exponential/cosine function'), it can be seen that when fitting, it is important to refer to domain experts when choosing functions to fit to the data. It may or may not make sense to try to fit a cosine function based on how oscillating global temperatures (and other variables that influence the size of the ice caps) are, or the exponential function may make less sense based on changes applied in global emission regulations (which would mean a further, continuing decrease is less likely). 

From the table above it can be seen that for 1984 the cosine function is relatively close.  As the the function leaves the data range, the values start to drift from the actual data. From this it could be deduced that a piecewise function using the cosine and other methods tested might be a better predicter.

**Conclusions**<br/>
We can see that some basis for the Vandermonde matrix for Interpolation with Polynomials are better conditioned then others. But the Interpolated function is still very sensitive to small changes in the input data. The LS methods used later are better, but could still give negative predictions for the Ice Area in the far future. This makes us think that the Cosine/Exponential functions are better, while giving similar results for the predictions of years 1984 and 2016-2018. 

The cosine functions would indicate that there is an equally periodic rise and fall in the ice levels, which we know is not completely true. It is not completely periodic, and due to human interference with the climate the trends observed now will not follow the general trends of the past.  

**Future work**<br/>
Since the fitting seems to be sensitive to the inital values when using the fit functions, it would be good to test with a wider range of intial values to determine the optimum starting position to fit the functions.  In addition it can be seen that some of the functions fit better for some parts of the data than others, therefore the use of a piecewise function might be benifiical.

Also some more testing with sensitivity to minor changes in input data could be done.